In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u432-ga~us1-0ubuntu2~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u432-ga~us1-0ub

In [ ]:
!pip install konlpy
!apt-get update
!apt-get install -y openjdk-11-jdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 42.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,113 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu 

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType

In [ ]:
# 1. SparkSession 생성
spark = SparkSession.builder \
    .appName("Job Recommendation System") \
    .getOrCreate()

# 2. 채용 공고 데이터 로드
job_postings = spark.read.csv("cleaned_data.csv", header=True)

In [ ]:
# 3. 텍스트 전처리
# tokenizer = Tokenizer(inputCol="requirement", outputCol="words")
# tokenized = tokenizer.transform(job_postings)

from konlpy.tag import Okt

def remove_stopwords_korean(text):
    okt = Okt()
    words = okt.nouns(text)
    # stopwords = ["은", "는", "이", "가", "을", "를", "에", "의", "와", "과", "로", "도", "에서", "으로", "그리고",",", "입니다",
                #  "안녕","(",")"]
    # return [word for word in words if word not in stopwords]
    return words
    ## 중복 제거
    #return list(set([word for word in words if word not in stopwords]))
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

remove_stopwords_udf = udf(remove_stopwords_korean, ArrayType(StringType()))

filtered = job_postings.withColumn("filtered", remove_stopwords_udf(col("requirement")))

ModuleNotFoundError: No module named 'konlpy'

In [ ]:
from pyspark.sql.functions import concat_ws

# 배열 컬럼을 문자열로 변환
filtered_for_csv = filtered.withColumn("filtered", concat_ws(" ", col("filtered")))
filtered_for_csv.show()

NameError: name 'filtered' is not defined

In [ ]:
import os

# Step 1: CSV 파일 저장 (여러 파티션 포함)
filtered_for_csv.coalesce(1).write.csv("./filtered_output", header=True, mode="overwrite")

# Step 2: 단일 CSV 파일로 병합 및 BOM 추가
output_dir = "./filtered_output"
output_file = "./filtered_output_bom.csv"

# 병합된 파일 찾기
for file_name in os.listdir(output_dir):
    if file_name.startswith("part-") and file_name.endswith(".csv"):
        part_file = os.path.join(output_dir, file_name)
        break

# BOM 추가 및 저장
with open(part_file, "r", encoding="utf-8") as original_file:
    content = original_file.read()

with open(output_file, "w", encoding="utf-8-sig") as bom_file:
    bom_file.write(content)

print(f"CSV with BOM saved to {output_file}")

CSV with BOM saved to ./filtered_output_bom.csv


In [ ]:
# 4. TF-IDF 계산
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(filtered)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)



In [ ]:
# 5. 사용자 입력 처리
user_input = "Python, Machine Learning, Data Analysis"  # 사용자 입력
user_df = spark.createDataFrame([(user_input,)], ["description"])
tokenized_user = tokenizer.transform(user_df)
filtered_user = remover.transform(tokenized_user)
featurized_user = hashingTF.transform(filtered_user)
user_features = idfModel.transform(featurized_user).select("features").first()["features"]



In [ ]:
# 6. 코사인 유사도 계산 함수
def cosine_similarity(v1, v2):
    dot_product = float(v1.dot(v2))
    norm_v1 = float(v1.norm(2))
    norm_v2 = float(v2.norm(2))
    return dot_product / (norm_v1 * norm_v2)

cosine_udf = udf(lambda x: cosine_similarity(user_features, x), FloatType())
similarity_df = rescaledData.withColumn("similarity", cosine_udf(col("features")))



In [ ]:
# 7. 추천 결과 정렬
recommended_jobs = similarity_df.select("job_id", "similarity").orderBy(col("similarity").desc())

# 8. 상위 N개 추천 출력
top_n = 5
recommended_jobs.show(top_n)

spark.stop()